In [14]:
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

In [2]:
zip_file_path = "/content/archive (2).zip"
extract_folder = "/content/dataset/"

In [3]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

os.listdir(extract_folder)

['yes', 'no']

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Definir transformações de pré-processamento para as imagens
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensiona as imagens
    transforms.ToTensor(),  # Converte a imagem em tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normaliza as imagens
])

In [6]:
data_dir = '/content/dataset/'

In [7]:
dataset = datasets.ImageFolder(data_dir, transform=transform)

In [12]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [13]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [15]:
class BrainTumorCNN(nn.Module):
    def __init__(self):
      super().__init__()

      self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 3 canais de entrada (RGB), 32 filtros
      self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 32 filtros, 64 filtros
      self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 64 filtros, 128 filtros

      self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Pooling 2x2

      self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Ajuste o número de neurônios dependendo do tamanho da imagem
      self.fc2 = nn.Linear(512, 2)  # 2 saídas (tumor ou sem tumor)

    def forward(self, x):
      x = self.pool(torch.relu(self.conv1(x)))  # Passa pela conv1 e aplica ReLU e MaxPool
      x = self.pool(torch.relu(self.conv2(x)))  # Passa pela conv2 e aplica ReLU e MaxPool
      x = self.pool(torch.relu(self.conv3(x)))  # Passa pela conv3 e aplica ReLU e MaxPool

      x = x.view(-1, 128 * 28 * 28)  # Achata o tensor para passar para a camada totalmente conectada
      x = torch.relu(self.fc1(x))  # Passa pela fc1 e aplica ReLU
      x = self.fc2(x)  # Passa pela fc2 (sem ativação, pois será usado com CrossEntropyLoss)

      return x

model = BrainTumorCNN()

In [19]:
model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-8.9242e-02,  9.9794e-03, -1.0425e-01],
                        [-1.8688e-01, -1.5869e-02, -1.0524e-01],
                        [ 7.7847e-02, -1.3926e-01, -1.4172e-01]],
              
                       [[-1.0412e-01, -1.5170e-02, -1.6354e-01],
                        [-1.2629e-01,  1.3341e-01, -1.8440e-01],
                        [ 5.6417e-02,  1.0178e-01,  1.2956e-01]],
              
                       [[-7.5893e-02, -1.5915e-01, -9.0480e-02],
                        [ 5.5465e-02,  1.6453e-01, -8.6509e-02],
                        [-4.4809e-02,  4.8279e-02,  4.4432e-03]]],
              
              
                      [[[ 1.5916e-01, -1.2351e-01,  1.8867e-01],
                        [-1.2072e-01, -1.0172e-01, -1.4391e-01],
                        [-1.1335e-01, -4.3323e-02, -1.4212e-01]],
              
                       [[-1.0825e-01,  4.0527e-02, -1.3183e-01],
                        [ 7.9680e-02, -4.1355

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")

        # Treinamento
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero gradientes
            optimizer.zero_grad()

            # Forward
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward + otimização
            loss.backward()
            optimizer.step()

            # Estatísticas
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f"Treinamento - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Validação
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)

        print(f"Validação - Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # Salvar o melhor modelo
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()

    # Carregar o melhor modelo
    model.load_state_dict(best_model_wts)
    return model

In [23]:
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 0/9
Treinamento - Loss: 0.7938 Acc: 0.5149
Validação - Loss: 0.6489 Acc: 0.5294
Epoch 1/9
Treinamento - Loss: 0.6010 Acc: 0.6089
Validação - Loss: 0.5698 Acc: 0.6078
Epoch 2/9
Treinamento - Loss: 0.5649 Acc: 0.7129
Validação - Loss: 0.4958 Acc: 0.8039
Epoch 3/9
Treinamento - Loss: 0.4731 Acc: 0.7921
Validação - Loss: 0.4335 Acc: 0.8039
Epoch 4/9
Treinamento - Loss: 0.4207 Acc: 0.8366
Validação - Loss: 0.4053 Acc: 0.8235
Epoch 5/9
Treinamento - Loss: 0.3564 Acc: 0.8465
Validação - Loss: 0.3928 Acc: 0.8235
Epoch 6/9
Treinamento - Loss: 0.2992 Acc: 0.8861
Validação - Loss: 0.4019 Acc: 0.8039
Epoch 7/9
Treinamento - Loss: 0.2473 Acc: 0.9257
Validação - Loss: 0.3888 Acc: 0.8039
Epoch 8/9
Treinamento - Loss: 0.1867 Acc: 0.9406
Validação - Loss: 0.5281 Acc: 0.8235
Epoch 9/9
Treinamento - Loss: 0.2194 Acc: 0.9059
Validação - Loss: 0.4085 Acc: 0.8627


In [24]:
model.eval()
val_corrects = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        val_corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

val_acc = val_corrects.double() / total
print(f"Acurácia no conjunto de validação: {val_acc:.4f}")

Acurácia no conjunto de validação: 0.8627
